# PepperOpenDayDemo

## Environment Setup

In [1]:
# built-in imports
# !pip install requests
# !pip install speechrecognition==3.8.1
# !pip install paramiko
# !pip install pandas
# !pip install beautifulsoup4
# !pip install pocketsphinx
# !pip install scipy

bumperDetector = ALProxy("ALTouch", constants.PEPPER_HOST, constants.PEPPER_PORT)

memory = ALProxy("ALMemory", constants.PEPPER_HOST, constants.PEPPER_PORT)

# bumperDetector.subscribe("Device/SubDeviceList/Platform/Back/Bumper/Sensor/Value")



touch = bumperDetector.subscriber("TouchChanged")

status = memory.getData("SoundDetected")

print(touch)

print(status)



self.touch.signal.connect(functools.partial(self.onTouched, "TouchChanged"))



def onTouched():

    print("touched")

bumperDetector = ALProxy("ALTouch", PEPPER_HOST, PEPPER_PORT)

touch = bumperDetector.session.subscribe("TouchChanged")

memory = ALProxy("ALMemory", PEPPER_HOST, PEPPER_PORT)

bumperDetector.session().service

dir(bumperDetector.session())

touch = memory.subscribeToEvent()

dir(bumperDetector.session)

In [1]:
### Naoqi Related Imports ###
import naoqi
from naoqi import qi
from naoqi import ALProxy

### System Related Imports ###
import fileinput, os, shutil, sys, random, threading, time

### Flask Related Imports ###
import json, requests, time

### Speech Recognition Imports ###
import pandas as pd
import paramiko
import speech_recognition as sr

/usr/local/lib/python2.7/site-packages/paramiko/transport.py:33: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography.hazmat.backends import default_backend


In [2]:
from prompts import *
import prompts
from fileTransfer import *
import fileTransfer as ft
from displayGeneration import *
import displayGeneration as dg
from interactiveControls import *
import interactiveControls as ic
from idle import *
import idle as idl
from humanInteraction import *
import humanInteraction as hi
from chatflow import convFlow


# ### Custom File Handing Imports ###
# from fileTransfer import *

# ### Custom Idle Behaviour Imports ###
# from idle import *

# ### Custom Page Display Imports ###
# from displayGeneration import *

# ### Custom Interaction, Behaviour and Display Imports ###
# from interactiveControls import *

# ### Custom Hard-Coded Prompt Imports ###
# from prompts import *

# ### Main Interaction Imports ###
# from humanInteraction import *

## Constants

In [3]:
### Networking Constants ###
GPT_HOST = "10.104.23.208"
PEPPER_HOST = "10.104.23.185"
GPT_PORT = 8891
PEPPER_PORT = 9559

### Audio Processing Constants ###
PEPPER_RECORDINGS_PATH = "/home/nao/microphones/recording.wav"
SERVER_RECORDINGS_PATH = "recordings/recording.wav"

### Display Generation Constants ###
FILE_NAME_TEMP = "display.html"
PEPPER_HTML_PATH = "/home/nao/.local/share/PackageManager/apps/robot-page/html/"  # page.html"
PEPPER_PAGE_LANDING = PEPPER_HTML_PATH + "page.html" # '/home/nao/.local/share/PackageManager/apps/robot-page/html/page.html'
PEPPER_QR_LANDING = PEPPER_HTML_PATH + "webfiles/qr.png"  # switched slash direction
PEPPER_IMG_LANDING = PEPPER_HTML_PATH + "webfiles/img.png"  # switched slash direction
TEXT_BY_ID_PATH = "textbyID.csv"

# Choregraph Behavior

In [4]:
import naoqi
from naoqi import qi
import time


def getBehaviors(behavior_mng_service):
    """
    Know which behaviors are on the robot.
    """
    names = behavior_mng_service.getInstalledBehaviors()
    print "Behaviors on the robot:"
    print names
    
    names = behavior_mng_service.getRunningBehaviors()
    print "Running behaviors:"
    print names
    
    names = behavior_mng_service.getDefaultBehaviors()
    print "Default behaviors:"
    print names
    

ip = '10.104.23.185'
port = '9559'

session = qi.Session()
session.connect("tcp://" + ip + ":" + port)
service = session.service("ALBehaviorManager")
# getBehaviors(service)


def launchAndStopBehavior(behavior_mng_service, behavior_name):
    """
    Launch and stop a behavior, if possible.
    """
    # Check that the behavior exists.
    if (behavior_mng_service.isBehaviorInstalled(behavior_name)):
        # Check that it is not already running.
        if (not behavior_mng_service.isBehaviorRunning(behavior_name)):
            # Launch behavior. This is a blocking call, use _async=True if you do not
            # want to wait for the behavior to finish.
            behavior_mng_service.runBehavior(behavior_name, _async=True)
            time.sleep(0.5)
        else:
            print "Behavior is already running."

    else:
        print "Behavior not found."
    return

    names = behavior_mng_service.getRunningBehaviors()
    print "Running behaviors:"
    print names

    # Stop the behavior.
    if (behavior_mng_service.isBehaviorRunning(behavior_name)):
        behavior_mng_service.stopBehavior(behavior_name)
        time.sleep(1.0)
    else:
        print "Behavior is already stopped."

    names = behavior_mng_service.getRunningBehaviors()
    print "Running behaviors:"
    print names


launchAndStopBehavior(service, '.lastUploadedChoregrapheBehavior/behavior_1')

## MAIN

In [ ]:
global idle
global responsesPipeline
global eyes

idle = idling()
eyes = EyesController()
eyes.startEyes("neutral")

try:
    track_head()

    while True:
        # Empty sentences list
        responsesPipeline = []
        
        # Idle behaviors to keep peppers temperature low
        idle.start_idle_behavior()

        # trigger smart listen
        detect()
        idle.stop()
        faceTracker()
        
        rCount = 0
        cCounter = 0
        while rCount < 3:
            query = listen()
            if query == '%low_volume_error%':
                speak("Sorry, I didn't quite catch that. Could you please repeat that?")
                rCount += 1
            else:
                pred_class, thresh = classifyQuery(query)

                if not thresh:
                    rcount += 1
                if pred_class == 'Courses':
                    cCounter += 1
                if cCounter == 2:
                    pred_class = 'SpecificCourse'


                repeat = convFlow(query, pred_class, thresh, responsesPipeline, eyes, rcount)

                if repeat:
                    rCount += 1

        
        # When the user has tried 3 times, return to default position
        return_to_default_pos()
        time.sleep(0.5)
        continue









        ### OLD FLOW ###

        # repeat = True
        # rcount = 0
        # # get course codes relevant to this
        # # if query search is not successful, try the query search again until it is successful
        # while (repeat and (rcount<4)):
        #     query = listen()
        #     repeat = queryCourseCodes(query, responsesPipeline, eyes)
        #     rcount += 1
            
        # if (rcount>=4):
        #     return_to_default_pos()

        #     time.sleep(0.5)

        #     continue
        
        # #take a pause here for users to read the tablet
        # time.sleep(2)
        
        # #ask if they need specific details about a course
        # repeat = True
        # rcount = 0
        # while repeat:
        #     query = listen()
        #     repeat = querySpecificCourse(query, responsesPipeline, eyes, rcount)
        #     rcount += 1
            
        # # give her some time after she has finished one person
        # time.sleep(5)
        # return_to_default_pos()
        
except Exception as e:
    print(e)
finally:
    stopListening()
    return_to_default_pos()
    idle.start_idle_behavior()
    print('Goodbye')

copyfile successful
Sent to Pepper
0.261222958565
Detect complete
copyfile successful
Sent to Pepper
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
[-15 -40 -20 ... 235 400 780]
<type 'numpy.ndarray'>
Saved noise reduced audio file to recordings/rn_recording.wav
('Listen complete, This is what I heard: ', u'what time')
copyfile successful
Sent to Pepper
copyfile successful
Sent to Pepper
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
[   100     70     75 ...  22940  -5220 -31945]
<type 'numpy.ndarray'>
Saved noise reduced audio file to recordings/rn_recording.wav

('Listen complete, This is what I heard: ', '%low_volume_error%')
copyfile successful
Sent to Pepper
Low volume error reached
copyfile successful
Sent to Pepper
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
[ -2

displayGeneration.py:182: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 182 of the file displayGeneration.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  target.replace_with(BeautifulSoup(subText))


Sent to Pepper
Sent to Pepper
copyfile successful
Sent to Pepper
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
[  40   45   50 ... -705 -700  165]
<type 'numpy.ndarray'>
Saved noise reduced audio file to recordings/rn_recording.wav
('Listen complete, This is what I heard: ', u'Bachelor of law Bachelor of cyber security')
('The Bachelor of Laws/Bachelor of Cyber Security program at Deakin University is a unique and comprehensive course that combines the study of law with cyber security skills to prepare students for careers within both fields.', 'D397')
copyfile successful
text match found
text match found
Sent to Pepper
Sent to Pepper
Sent to Pepper
copyfile successful
Sent to Pepper
0.354666620493
Detect complete
copyfile successful
Sent to Pepper
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
[  35   25   -5 ...   85  580 1070]
<type 'numpy.